In [6]:
import numpy as np
# netCDF4 needs to be installed in your environment for this to work
import xarray as xr
import rioxarray as rio

In [ ]:
nc_path = r"C:\Users\VMO\Downloads\VHP.G04.C07.npp.P2015001.SM.nc"

In [ ]:
# Open as xarray
ds  = xr.open_dataset(nc_path)
ds

In [ ]:
# Check if 'lat' and 'lon' already exist in the dataset and rename them if necessary
if 'lat' in ds:
    ds = ds.rename({'lat': 'lat_old'})
if 'lon' in ds:
    ds = ds.rename({'lon': 'lon_old'})

# Generate the latitude and longitude values
lat_values = np.linspace(75.024, -55.152, ds.dims['ny'])
lon_values = np.linspace(-180.0, 180.0, ds.dims['nx'])

# Assign these coordinates to the dataset
ds = ds.assign_coords(lat=('ny', lat_values), lon=('nx', lon_values))
ds = ds.swap_dims({'ny': 'lat', 'nx': 'lon'})

# Now proceed with flipping and scaling steps

In [ ]:
ds = ds.reindex(lat=list(reversed(ds['lat'])))

In [ ]:
for var in ds.data_vars:
    attrs = ds[var].attrs
    if 'add_offset' in attrs and 'scale_factor' in attrs:
        scaled_integer = ds[var]
        add_offset = attrs['add_offset']
        scale_factor = attrs['scale_factor']
        ds[var] = (scaled_integer - add_offset) * scale_factor
    else:
        print(f"{var} does not have both 'add_offset' and 'scale_factor' attributes.")

In [ ]:

# Extract the SMN variable from the dataset
smn_data = ds['SMN']

# Set spatial dimensions, CRS, and save to GeoTIFF
smn_data.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
smn_data.rio.write_crs("EPSG:4326", inplace=True)
output_file_path_smn = r"C:\Users\VMO\Desktop\output_filename_smn.tif"
smn_data.rio.to_raster(output_file_path_smn)


In [ ]:

# Extract the SMN variable from the dataset
smt_data = ds['SMT']

# Set spatial dimensions, CRS, and save to GeoTIFF
smt_data.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
smt_data.rio.write_crs("EPSG:4326", inplace=True)
output_file_path_smt = r"C:\Users\VMO\Desktop\output_filename_smt.tif"
smt_data.rio.to_raster(output_file_path_smt)

In [8]:
# Step 1: Read the raster data
with rio.open(r'C:\Users\VMO\Desktop\output_filename_smt.tif', 'r') as src:
    # Read the data
    data = src.read(1)  # assuming you're reading the first band; adjust as necessary
    
    # Step 2: Scale the data
    data = data / 1000.0
    
    # Set up the metadata for writing
    out_meta = src.meta.copy()
    
    # Step 3: Save the scaled data to a new raster file
    with rio.open(r'C:\Users\VMO\Desktop\output_filename_smt_scaled.tif', 'w', **out_meta) as dest:
        dest.write(data, 1)  # writing the scaled data to the first band

AttributeError: module 'rioxarray' has no attribute 'open'